In [1]:
# Importem llibreries
import requests
import pandas as pd
from bs4 import BeautifulSoup


# Definim la url en la que fer la lectura
root_url = "https://www.barcelona-airport.com"
arrivals = "/eng/arrivals.php"
url = root_url + arrivals


'https://www.barcelona-airport.com/eng/arrivals.php'

In [2]:
# Trobem els enllaços de cada vol a la url examinada
page = requests.get(url)
soup = BeautifulSoup(page.content)
fnum = soup.find_all(id="fnum")
links = []
for fn in fnum[2:]:
    a = fn.find('a')
    link = str(a['href'])
    links.append(link)


In [79]:
# Creem un llistat en blanc. 
flights  = []

# Per a cada enllaç entrem i busquem els valors dins la pàgina per obtenir les dades desitjades de cada vol
for link in links:
    
    URL = root_url + link
    Page = requests.get(URL)
    Soup = BeautifulSoup(Page.content)
    
    codecomp0 = Soup.find(id="flight_header2")
    if codecomp0 is None:
        codecomp = Soup.find(id="flight_info").h2.get_text()
        status   = Soup.find(id="flight_info").h2.parent.find_next_sibling().h2.get_text()
    else:
        codecomp = codecomp0.get_text()
        status   = codecomp0.parent.find_next_sibling().h2.get_text()
    code     = codecomp[:codecomp.find('(')-1].replace(" ", "")
    company  = codecomp[(codecomp.find('(')+1):(len(codecomp)-1)]

    origin   = Soup.find(id='flight_dep').h3.get_text()
    Dep      = Soup.find(id='flight_dep').get_text()
    first_app  = Dep.find("Date: ")+6
    second_app = Dep.find(":", first_app)+1
    third_app  = Dep.find(":", second_app)+1
    fourth_app = Dep.find(":", third_app)+1
    fifth_app  = Dep.find(":", fourth_app)
    depDate  = Dep[first_app:first_app+10].strip().replace(" ","")
    depTime  = Dep[second_app:third_app+2].strip().replace(" ","")
    sDepTime = Dep[fourth_app:Dep.find("Terminal:")].strip().replace(" ","")
    depTerm  = Dep[Dep.find("Terminal: ")+10:Dep.find("Gate:")].strip().replace(" ","")
    depGate  = Dep[Dep.find("Gate:")+6:].strip().replace(" ","")

    destin   = Soup.find(id='flight_arr').h3.get_text()
    Arr      = Soup.find(id='flight_arr').get_text()
    first_app  = Arr.find("Date: ")+6
    second_app = Arr.find(":", first_app)+1
    third_app  = Arr.find(":", second_app)+1
    fourth_app = Arr.find(":", third_app)+1
    fifth_app  = Arr.find(":", fourth_app)+1
    
    arrDate  = Arr[first_app:first_app+10].strip().replace(" ","")
    arrTime  = Arr[second_app:third_app+2].strip().replace(" ","")
    sArrTime = Arr[fourth_app:fifth_app+2].strip().replace(" ","")
    
    arrTerm  = Arr[Arr.find("Terminal:")+10:Arr.find("Gate:")].strip().replace(" ","")
    arrGate  = Arr[Arr.find("Gate:")+6:Arr.find("Gate:")+10].strip().replace(" ","")
    plane    = str(Soup.find(id='flight_other'))
    plane    = plane[plane.find('Plane:')+6:].strip()
    if "<br" in plane:
        plane = plane[:plane.find("<br")].strip()
    if "</d" in plane:
        plane = plane[:plane.find("</d")].strip()

    # Cada valor extret passa a formar part d'una tupla
    flight = (URL, code, company, status, plane,
              origin, depDate, depTime, sDepTime, depTerm, depGate,
              destin, arrDate, arrTime, sArrTime, arrTerm, arrGate)
    
    # Unim totes les tuples en la llista que s'ha inicialitzat al abans del loop
    flights.append(flight)

    

In [83]:
# Indiquem els noms de les columnes del nostre dataset
columns = ['link', 'code', 'company', 'status', 'plane', 'origin', 'depDate', 'depTime', 'sDepTime', 'depTerm', 'depGate',
              'destin', 'arrDate', 'arrTime', 'sArrTime', 'arrTerm', 'arrGate']

# El transformeme en un dataframe i l'exportem en format CSV
data = pd.DataFrame(flights, columns = columns)
data.to_csv('prova.csv', index = False)